# Running the model on data
## This is where we can truly get wild and rowdy with our algorithm

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import keras
from keras import Model
import PIL
from PIL import Image, ImageFilter
import numpy as np
import os
import pathlib
import glob

In [2]:
print("Tensorflow version: ", tf.__version__)
print("GPU Availability: ", tf.config.list_physical_devices('GPU'))

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=7500)])
    except RuntimeError as e:
        print(e)


Tensorflow version:  2.3.0
GPU Availability:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Get the model
## Make sure this architecture matches the one you trained

In [3]:
inputs = layers.Input((512,512,15))
x = layers.Conv2D(64, (7,7), padding='same', activation='relu')(inputs)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Conv2D(32, (5,5), padding='same', activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Conv2D(16, (3,3), padding='same', activation='relu')(x)
skip_one = x
x = layers.MaxPooling2D(pool_size=(2,2))(x)


x = layers.BatchNormalization(axis=-1)(x)
x = layers.Conv2D(32, (7,7), padding='same', activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Conv2D(32, (5,5), padding='same', activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Conv2D(16, (3,3), padding='same', activation='relu')(x)
skip_two = x
x = layers.MaxPooling2D(pool_size=(2,2))(x)

x = layers.BatchNormalization(axis=-1)(x)
x = layers.Conv2D(32, (5,5), padding='same', activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Conv2D(32, (3,3), padding='same', activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
encoded = layers.Conv2D(16, (1,1), padding='same', activation='relu')(x)


x = layers.Conv2D(32, (3,3), padding='same', activation='relu')(encoded)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Conv2DTranspose(32, (5,5), padding='same', strides=2, activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)

x = layers.Concatenate(axis=-1)([x, skip_two])
x = layers.Conv2D(32, (3,3), padding='same', activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Conv2D(32, (5,5), padding='same', activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Conv2DTranspose(32, (7,7), padding='same', strides=2, activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)

x = layers.Concatenate(axis=-1)([x, skip_one])
x = layers.Conv2D(32, (3,3), padding='same', activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Conv2D(32, (5,5), padding='same', activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Conv2D(32, (5,5), padding='same', activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
decoded = layers.Conv2D(3, (7,7), padding='same', activation='sigmoid', dtype='float32')(x)

ae = keras.Model(inputs, decoded)


In [4]:
ae.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 15 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 64) 47104       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 512, 512, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 32) 51232       batch_normalization[0][0]        
_______________________________________________________________________________________

# This is the best part of the code
## You can really let loose and throw a bunch of garbage into the model and see what happens. You'll get a lot of boring stuff, and maybe you'll get something cool.
### I've already given you a huge headstart with the "history" thing-- if you want actual results that look somewhat realistic, that parameter might be your best friend.


# VERY IMPORTANT NOTE:
## If you come up with any cool way to mess with this part of the code you gotta let me know lmao

In [92]:
# Load weights
ae.load_weights('logs/models/resid_road_predictor_0054')

# Get some frames to play with
sampled_frames = glob.glob('/home/tom/data/frames/road_frames_sampled_holder/road_frames_sampled/*')
sampled_frames.sort()

# Which frame are we starting at, boys?
start_idx = 1000
# Here you can see I'm running the model on data that's
# bigger than what it was trained on
prediction_array = np.zeros([1024,1024,15])
# This loop preps the data.
for i in range(5):
    img = Image.open(sampled_frames[start_idx+i]).resize([1024,1024])
    prediction_array[:,:,i*3:(i+1)*(3)] = np.array(img)/255

next_frame = np.expand_dims(prediction_array,0)
true_frames = np.zeros((1024, 1024, 15))
# 0 <= history <= n_frames (5 here)
# Higher history -> More realistic results
# Gives the model {history} many ground truth frames and
# {n_frames - history} many generated frames
history = 2
# c stands for constant
# Just used for messing around, nothing of importance
c = -1
for i in range(500):
    print(i)
    # Get True History
    for j in range(5):
        img = Image.open(sampled_frames[start_idx+i+j+1]).resize([1024,1024])
        true_frames[:,:,j*3:(j+1)*(3)] = np.array(img)/255
    
    # Make Prediction
    next_frame = ae.predict(next_frame)
    
    # Save prediction to show
    toshow = next_frame
    # Set all the frames back one
    prediction_array[:,:,:12] = prediction_array[:,:,3:]
    # Recover history
    prediction_array[:,:,:3*history] = \
            true_frames[:,:,:3*history]
    
    # The next two if/else loops solely exists to mess with our inputs
    # to try and get cool artifacts.
    # If you just want to run the model with no funnybusiness, use:
    '''prediction_array[:,:,12:] = next_frame'''
    if i % 10 == 0:
        c*=-1
    else:
        c*=1
    if i % 1 == 0:
        prediction_array[:,:,12:] = (c)*next_frame + \
                                (-1*c)*np.array(Image.open(\
                                                sampled_frames[start_idx+5+i]).resize((1024,1024)))/255 +\
                                 (0.1)*np.random.normal(size=(1024,1024,3))
    else:
        prediction_array[:,:,12:] = (1)*next_frame
    
    # Save the image, maybe show it too
    next_frame = np.expand_dims(prediction_array,0)
    Image.fromarray((toshow[0]*255).astype('uint8')).save('logs/hires/frame%04d.jpg'%i)
    if i % 50 == 0:
        Image.fromarray((toshow[0]*255).astype('uint8')).show()
        
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27